## Autonomous Mobile Robot Pallet Truck inside a Smart Factory

### Table of contents
 - [1. Overview](#1.-Overview)
 - [2. Gazebo simulation](#2.-Gazebo-simulation)
     - [2.1 AMR Pallet Truck](#2.1-AMR-Pallet-Truck)
         - [2.1.1 amr_description](#2.1.1-amr_description)
         - [Geometry](#Geometry)
         - [Actuators](#Actuators)
         - [Sensors](#Sensors)
     - [2.2 Gazebo environment](#2.2-Gazebo-environment)
       - [2.2.1 Gazebo models](#2.2.1-Gazebo-models)
         - [CNC machine models](#CNC-machine-models)        
         - [Chip bin model](#Chip-bin-model)
 -  [3. Navigation](#3.-Navigation)
   - [3.1 amr_mapping](#3.1-amr_mapping)
   - [3.2 amr_localization](#3.2-amr_localization)
   - [3.3 amr_navigation](#3.3-amr_navigation)
       - [Global planner](#Global-planner)
       - [Local planner](#Local-planner)
 - [4. Chip bin management](#4.-Chip-bin-management)
   - [4.1 amr_detection](#4.1-amr_detection)
   - [4.2 amr_rendezvous](#4.2-amr_rendezvous)
   - [4.3 amr_disengagement](#4.3-amr_disengagement)
   - [4.4 amr_webpage](#4.4-amr_webpage)
 - [5. Installation](#5.-Installation)
 - [6. Usage](#6.-Usage)


 
### 1. Overview

The four industrial revolution (4IR) pushes the manufacturing industry to an integration with newest technologies like the Artificial Intelligence, the additive manufacturing, the augmented reality and the IoT Internet of Things. The 4IR also includes an advancement in connectivity between machines, robots, sensors, etc. that gives rise to “smart factories”.

The following project refers to a smart factory in which the IoT, the advanced connectivity and the ROS technology allow an autonomous management of the raw and waste materials. In particular, the environment, shown in the figure 1.1, reproduces a mechanical workshop for working metals with numerical control machines; where an AMR pallet truck ensures the handling of both raw or semi-finished pieces and the transport of residual metal shaving.

<img src="images/mechanical_workshop.png" width="700" />

Figure 1.1: Mechanical workshop environment. (1) AMR pallet trucks, (2) CNC machine, (3) full chip bin, (4) empty chip bin.

In the following scenario, it is staged an example of management of the residual metal chip bin by the AMR pallet truck using a web interface, as shown in figure 1.2. It leaves the fully automatic management to a later implementation.

<img src="images/webinterface.png" width="700" />

Figure 1.2: Web Interface. Buttons: (1) go to full bin, (2) fork the bin, (3) elevator up, (4) go to recycling, (5) elevator down, (6) disengagement bin, (7) recycling bin, (8) go to empty bin, (9) go to CNC machine, (10) go home, (11) move forward, (12) move backward, (13) move turn CW, (14) move turn CCW, (15) stop, (16) connect/disconnect. Readings: (17) odometry position, (18) obstacle distances.

The project, created using ROS Noetic and Python language, has been divided into four main parts:
 - Gazebo simulation 
 - chip bin management
 - navigation 
 - usage

The first part concerns the construction of the AMR and its environment, i.e. the mechanical workshop with CNC machines and the chip bins. The second one is dealing with the actions of the AMR, like: the detection, the engagement and the disengagement of the chip bin. The third regards the transport of both empty and full europallets. The last one is an explanation on how to do a chip bin replacement cycle using a web interface. To achive these objectives 9 packages were created, grouped as follows:

 - **Gazebo simulation** 
     - amr_description
     - amr_environment     
 - **chip bin management**
      - amr_detection
      - amr_rendezvous
      - amr_disengagement  
      - amr_webpage
 - **navigation**
      - amr_mapping
      - amr_localization
      - amr_navigation
 - **usage**
      
      

<img src="images/rqt_graph.png" width="700" />

Figure 1.3.: rqt_graph of the complete project (all nodes and topics).

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 2. Gazebo simulation
#### 2.1 AMR Pallet Truck
The creation of the AMR Pallet Truck has been made inside the amr_description package.
#### 2.1.1 amr_description
The amr_description package is organized in the following folders structure:
- config
  - amr_controller.yaml
- launch
  - amr_joints_control.launch
  - amr_urdf_visualize.launch
  - elevator_service_server.launch
  - main_project.launch
  - rosbridge.launch
  - spawn_main_amr_empty_world.launch
  - spawn_main_amr_warehouse.launch
- models
  - meshes
   - amr_blender.dae
   - amr_fork.dae
   - amr_wheel.dae
- rviz
  - rviz_config.rviz
- src
  - scripts
    - elevator_service_server.py
- srv
  - ElevatorServiceMessage.srv
- urdf
  - amr_gen.pdf
  - amr.gv
  - amr.pdf
  - amr.xacro
- worlds
  - empty.world

The *urdf* folder contains the file *amr.xacro* which is a virtual description of the AMR in terms of *geometry*, *sensors* and *actuators*.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### Geometry
The AMR pallet truck, of figure 2.1, has a mass of 90 kg and occupies an area of 1 x 1.2 metres. It is equipped with:
- two differential driving wheels;
- two caster wheels;
- one front laser scanner (FOV 180 degree);
- one rear laser scanner (FOV 60 degree);
- two lifting forks.

The design of a first virtual model was created in **PTC Creo Parametric CAD**, inspired by a manual pallet truck. The CAD model is shown in figure 2.1.


The **XACRO** technology allows to use parameters and the **MACROS** is useful when it needs to repeat parts like the driving wheels and the caster wheels. Also, the parameters enable to modify mass, moments of inertia, morphology, etc. quickly and without entering the code. In particular, the geometric parameters values, see code 2 below,  have been obtained from the CAD, figures 2.1 and 2.2. A short excerpt follows:

<img src="images/amr_pallet_truck_creo.png" width="700" />

Figure 2.1: CAD model of the AMR Pallet truck, (1) emergency stop button, (2) flashing lamp, (3) differential driving wheels, (4) lifting forks, (5) caster wheels, (6) chassis.

<img src="images/amr_drawing.png" width="700" />

Figure 2.2:AMR main dimensions, [mm]

From PTC Creo, each piece of figure 2.1, was exported in *.stl* format to the *Blender* environment where the colors were added and finally exported again in *.dae* format, see figure 2.3 

<img src="images/chassis_blender.png" width="700" />

Figure 2.3: Blender environment. (6) chassis.

In [ ]:
    <xacro:macro name="driving_wheel_link" params="number colour">
        
        <link name="driving_wheel_M${number}_link">    
            
            <visual>
                <origin xyz="0 0 0" rpy="0 0 0"/> 
                <geometry>
                    <mesh filename="package://amr_description/models/meshes/amr_wheel.dae"/>
                </geometry>
                <material name="blue"/>
            </visual>  

            <collision>
                <origin xyz="0 0 0" rpy="0 0 0"/>
                <geometry>
                    <cylinder length="${dwt}" radius="${dwr}"/>
                </geometry>
            </collision>  

            <inertial>
                <origin xyz="0 0 0" rpy="0 0 0"/>            
                <mass value="${dwm}" />
                <inertia ixx="${dw_ixx}" ixy="0.0" ixz="0.0" iyy="${dw_iyy}" iyz="0.0" izz="${dw_izz}"/>
            </inertial>        

        </link>

        <gazebo reference="driving_wheel_M${number}_link">
            <kp>25000.0</kp> <!-- This coefficient sets the static contact stiffness -->
            <kd>25000.0</kd> <!-- This coefficient sets the dynamic contact stiffness -->
            <mu1>10.0</mu1> <!-- The static friction coefficient -->
            <mu2>10.0</mu2> <!-- The dynamic friction coefficient -->
            <!--material>Gazebo/${colour}</material--> <!-- to enable the color of .dae files -->
        </gazebo>

    </xacro:macro>

Code 2.1: MACROS defining the driving wheels link

The tags in the code above are needed to create a model in Gazebo. With the tags **< visual >** and **< collision >** are defined respectively: the appearance via the meshes .dae file, and the force reaction cylinder, with the parameters *dwt* and *dwr* specified below. At the end, the tag **< gazebo reference >** allows to add physical properties between the driving_wheel_link and the floor. Note that to enable color management in .dae file, it needs comment the **< material >** tag. Other details are specified as comments in the code above.

In [ ]:
    <!-- driving wheels property -->
    <xacro:property name="dwm" value="0.6" />                                  
    <xacro:property name="dwd" value="0.200" />                                
    <xacro:property name="dwr" value="${dwd/2}" />                            
    <xacro:property name="dwt" value="0.044" />              

Code 2.2: dwm: driving wheel mass, [kg], dwd: driving wheel diameter, [m] , dwr: driving wheel radius, [m], dwt: driving wheel tread, [m].

In [ ]:
    <xacro:property name="dw_ixx" value="${(dwm/12.0)*(3*dwr*dwr+dwt*dwt)}" />  
    <xacro:property name="dw_iyy" value="${dw_ixx}" />                         
    <xacro:property name="dw_izz" value="${(dwm*dwr*dwr)/2.0}" />                      

Code 2.3: dw_ixx: driving wheel inertia x axis, [kgm^2] , dw_iyy: driving wheel inertia y axis, [kgm^2], dw_izz: driving wheel inertia z axis, [kgm^2].

Furthermore, the geometry of the AMR is built using the **< link >** and **< joint >** tags respecting a logic of parent/child relationships. The joints of the driving wheels and the caster wheels are of the *revolute* type, instead the joints of the lifting forks are of the *prismatic* type. 

Finally, the XACRO file was verified using the following tool.

In [ ]:
xacro amr.xacro > tmp.urdf && check_urdf tmp.urdf && rm tmp.urdf

To easily verify the correctness of the connections made (geometry) the amr_gen.urdf file is visualized, as follows:

In [ ]:
xacro amr.xacro > amr_gen.urdf
urdf_to_graphiz amr_gen.urdf

<img src="images/link_joint_connections.png" width="700" />

Figure 2.4: overall virtual model of the AMR with link-joint connections.

<img src="images/link_joint_connections_detail.png" width="700" />

Figure 2.5: detail of the AMR virtual model. Rectangular and elliptical shapes represent respectively links (frames) and joints.

The figure 2.6 shows the transformations (arrows) between the links (ellipsis). It allows to graphically verify the correctness of the trasformations with the following command:

In [ ]:
rosrun rqt_tf_tree rqt_tf_tree

<img src="images/tf_tree.png" width="700" />

Figure 2.6: Transformation tree of AMR Pallet Truck: (1) base_link, (2) front_laserscanner_link, (3) rear_laserscanner_link, (4) caster_wheel_M1_link, (5) wheel_M1_link, (6) caster_wheel_M2_link, (7) wheel_M2_link, (8) driving_wheel_M1_link, (9) driving_wheel_M2_link, (10) elevator_dx_link, (11) elevator_sx_link, (12) footprint.

Finally, the AMR visualized in RViz appears as shown in figure 2.7. To verify the movement and the limit values of the joints, the AMR is displayed in RViz launching the **joints publisher GUI**, the figure 2.8 shows the movement of the prismatic joints of the lifting forks.

<img src="images/amr_link_rviz.png" width="700" />

Figure 2.7: Visualization of the AMR's link and trasformations in Rviz. (1) front_laserscanner_link, (2) driving_wheel_M1_link, (3) rear_laserscanner_link, (4) elevator_sx_link, (5) caster_wheel_M1_link, (6) wheel_M1_link

<img src="images/amr_joint_gui.png" width="700" />

Figure 2.8: Visualization of the AMR with meshes in Rviz

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### Actuators
The AMR pallet truck has four actuators: two motors for the differential wheels and two linear actuators for the lift forks. The choice of two different actuators for the lift forks has been done to allow an indipendent movement for next developements.

The motors, by the differential drive plugins, are applied to the two revolute joints well positioned as they appear in figure 2.7. While the linear actuators are managed by the plugin below.

In [ ]:

    <gazebo>
        <plugin name="gazebo_ros_control" filename="libgazebo_ros_control.so">
            <robotNamespace>/amr</robotNamespace>
        </plugin>
    </gazebo>
    
   <joint name="base_link_elevator_sx_link_joint" type="prismatic">  
        <parent link="base_link"/>
        <child link="elevator_sx_link"/>
        <origin rpy="0 0 0" xyz="-0.85 0.186 -0.040"/> <!--respect to parent axis -->
        <limit effort="1000.0" lower="${flp}" upper="${fup}" velocity="${fev}"/>        
        <axis xyz="0 0 1"/>
   </joint>

    <transmission name="tran1">
        <type>transmission_interface/SimpleTransmission</type>
        <joint name="base_link_elevator_sx_link_joint">
            <hardwareInterface>hardware_interface/EffortJointInterface</hardwareInterface>
        </joint>
        <actuator name="motor1">
            <hardwareInterface>hardware_interface/EffortJointInterface</hardwareInterface>
            <mechanicalReduction>${ftr}</mechanicalReduction>
        </actuator>
    </transmission>

Code 2.4: Gazebo code for the actuators.

The control config YAML file is used to create the transmission with the PID control.

In [ ]:
amr:
    # Publish all joint states -----------------------------------
    joint_state_controller:
      type: joint_state_controller/JointStateController
      publish_rate: 50

    # Forks Position Controllers ---------------------------------------
    base_link_elevator_sx_link_joint_position_controller:
      type: effort_controllers/JointPositionController
      joint: base_link_elevator_sx_link_joint
      pid: {p: 100000.0, i: 100.0, d: 100}       

    # Forks Position Controllers ---------------------------------------
    base_link_elevator_dx_link_joint_position_controller:
      type: effort_controllers/JointPositionController
      joint: base_link_elevator_dx_link_joint
      pid: {p: 100000.0, i: 100.0, d: 100}




Code 2.5: Controller parameters

The PID parameters have been configured with Dynamic Reconfiguration panel.

In [ ]:
rosrun rqt_gui rqt_gui

Two useful plugins like **Message Publisher** and **Dynamic Reconfiguration** allow you to set the values of the fork joint PID controllers.

<img src="images/amr_joint_dynamic_reconfiguration.png" width="700" />

Figure 2.9: Dynamic reconfiguration panel for PID joint controller setup.

In [ ]:
<!-- * * * driving_wheel_links * * * -->
    <xacro:driving_wheel_link number="1" colour="Blue" />
    <xacro:driving_wheel_link number="2" colour="Blue"/>

    <joint name="base_link_driving_wheel_M1_joint" type="continuous">
        <parent link="base_link"/>
        <child link="driving_wheel_M1_link"/>
        <origin xyz="0 ${-clad/2+dwt/2} ${0}" rpy="1.57 0 0"/> <!-- <origin xyz="0 ${-clad/2} ${-cwlh}" rpy="1.57 0 0"/> * * * -->
        <limit effort="15.0" velocity="0.1"/>
        <axis xyz="0 0 -1"/>
    </joint>


    <joint name="base_link_driving_wheel_M2_joint" type="continuous">
        <parent link="base_link"/>
        <child link="driving_wheel_M2_link"/>
        <origin xyz="0 ${clad/2-dwt/2} ${0}" rpy="1.57 0 0"/>
        <limit effort="15.0" velocity="0.1"/>
        <axis xyz="0 0 -1"/>
    </joint>


    <gazebo>
        <plugin name="differential_drive_controller" filename="libgazebo_ros_diff_drive.so">

            <alwaysOn>true</alwaysOn>
            <updateRate>50</updateRate>
            <leftJoint>base_link_driving_wheel_M2_joint</leftJoint>
            <rightJoint>base_link_driving_wheel_M1_joint</rightJoint>
            <wheelSeparation>${clad+dwt}</wheelSeparation>
            <wheelDiameter>${dwd}</wheelDiameter>
            <wheelAcceleration>0.1</wheelAcceleration>
            <wheelTorque>30</wheelTorque>
            <commandTopic>cmd_vel</commandTopic>
            <odometryTopic>odom</odometryTopic>
            <odometryFrame>odom</odometryFrame>
            <robotBaseFrame>base_footprint</robotBaseFrame>
            <publishWheelTF>true</publishWheelTF>
            <publishOdom>true</publishOdom>
            <robotBaseFrame>base_link</robotBaseFrame>
            <publishWheelJointState>true</publishWheelJointState>
            <legacyMode>false</legacyMode>
            <rosDebugLevel>false</rosDebugLevel>
            <publishOdomTF>true</publishOdomTF> <!--TF transforms to visualize in RVIZ all the positions of the wheels. -->
            <odometrySource>world</odometrySource>
            <publishTf>1</publishTf>
        </plugin>
    </gazebo>  

Code 2.6: Differential drive controller plugin.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### Sensors
The AMR has two laser scanners, as shown in figure 2.7. In this, position 1 shows the front laser with a 180 degree field of view (FOV) set up for the navigation stack. Position 3 shows the second laser scanner which supplies the data to carry out the detection phase of the europallets (chip bins). The detection strategy is to find the placement of the europallet by measuring the distances and angles between the laser and two reflectors fixed to the ends of the europallet itself.


<img src="images/rear_laser_gazebo.png" width="700" />

Figure 2.10: europallet detection phase. (1) left reflector, (2) right reflector, (3) rear laser scanner with 60 degree FOV.

<img src="images/rear_laser_rviz.png" width="700" />

Figure 2.11: europallet detection phase. (1) left reflector frame, (2) right reflector frame, (3) rear laser scanner frame.

A 60 degree FOV has been set to have more resolution and to obtain more precise and stable measurements and frames. The code below is the controller plugin that publishes the readings into /rear_scan topic. Details on the front laser scanner are similar and have not been reported.

In [ ]:
 <!-- Sensor Laser in the back -->

    <link name="rear_laserscanner_link">
        
        <visual>
            <origin rpy="0.0 0 0" xyz="0 0 0"/>
            <geometry>
                <box size="0.0005 0.0005 0.0005"/>
            </geometry>
            <material name="green"/>
        </visual>

        <collision>
            <origin rpy="0.0 0 0" xyz="0 0 0"/>
            <geometry>
                <box size="0.0005 0.0005 0.0005"/>
            </geometry>
        </collision>

        <inertial>
            <mass value="1e-5" />
            <origin xyz="0 0 0" rpy="0 0 0"/>
            <inertia ixx="1e-6" ixy="0" ixz="0" iyy="1e-6" iyz="0" izz="1e-6" />
        </inertial>

    </link>

    <joint name="base_rear_laserscanner_joint" type="fixed">
        <parent link="base_link"/>
        <child link="rear_laserscanner_link"/>
        <origin xyz="${-clod/2} 0 ${flsh}" rpy="0 0 1.57"/> <!-- <origin xyz="${-clod/2} ${-clad/2} ${flsh}" rpy="0 0 1.57"/> -->
    </joint>

    <gazebo reference="rear_laserscanner_link">
        <sensor type="ray" name="rear_rplidar_sensor">
            <pose>0 0 0 0 0 0</pose>
            <visualize>true</visualize>
            <update_rate>40</update_rate>
            <ray>
                <scan>
                    <horizontal>
                        <samples>720</samples>
                        <resolution>1</resolution> <!-- step that corresponds to an angle -->
                        <min_angle>1.2217</min_angle>
                        <max_angle>1.9199</max_angle>
                    </horizontal>
                </scan>
                <range>
                    <min>0.10</min>
                    <max>5.0</max>
                    <resolution>0.01</resolution> <!-- distance in [m] -->
                </range>
                <noise>
                    <type>gaussian</type>
                    <!-- Noise parameters based on published spec for Hokuyo laser
                         achieving "+-30mm" accuracy at range < 10m.  A mean of 0.0m and
                         stddev of 0.01m will put 99.7% of samples within mu+3*sigma and mu-3*sigma
                         that is 0+3*0.01 = +0.03 and -0.03m of the true readings. -->
                    <mean>0.0</mean>
                    <stddev>0.01</stddev>
                </noise>
            </ray>
            <plugin name="gazebo_ros_head_hokuyo_controller" filename="libgazebo_ros_laser.so">
                <topicName>rear_scan</topicName>
                <frameName>rear_laserscanner_link</frameName>
            </plugin>
        </sensor>
    </gazebo>

Code 2.7: Rear laser scanner controller plugin publishes range data through sensor_msgs::LaserScan ROS topic.

The URDF file is loaded to the **param server** variable called "robot_description." 
Start the jointstate publisher and the robotstate publisher. These will publish the TFs of the URDF of the AMR links and joints.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 2.2 Gazebo environment
This package takes care of building  the environemnt in which the AMR moves and interacts. Its structure is the following:

 - amr_environemnt
   - launch
     - no_roof_small_warehouse.launch
   - models
     - aws_robomaker_warehouse_GroundB_01
     - aws_robomaker_warehouse_Lamp_01
     - aws_robomaker_warehouse_PalletJackB_01
     - aws_robomaker_warehouse_RoofB_01
     - aws_robomaker_warehouse_WallB_01
     - becket_model
     - cncmillingbig_model
     - cncmillingsmall_model
     - full_becket_model
     - person_standing
   - src
   - worlds
      - no_roof_small_warehouseworld
   - CmakeLists.txt
   - Package.xml

The contents of the model folders were partly created ex novo and partly using resources found on the net. In particular, the components such as the **walls**, the **floor**, the **lamp** and the **person standing** are those of the Amazon warehouse package available at https://github.com/aws-robotics/aws-robomaker-small-warehouse-world and the Gazebo library. Instead the **CNC machines** and the **chip bins (full and empty) on europallets** were created from scratch.


[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 2.2.1 Gazebo models
The main objects that make up the environment in which the AMR moves are:
- CNC machines;
- Chip bins on europallets.

#### CNC machine models
In the list above, **cncmillingbig_model** and **cncmillingsmall_model** models are contained in folders with the following structure:

- cncmillingbig_model
   - materials
       - textures
   - meshes
     - VF-11_50_STEP__7_2022_low_r_03.dae
- model.config
- model.sdf


- cncmillingsmall_model
   - materials
       - textures
   - meshes
     - VF-1_STEP_11_2021_low.dae
- model.config
- model.sdf

The model.sdf and the model.config files are reported in the Codes 2.8 and 2.9 below for the big machine only. The fist one contains all the data to enable the CNC machines to interact in an physical environment with the AMR, like: mass, moment of inertia, collision and appearence.

About the appearence, the step files of VF1 and VF11-50 CNC machines have been downloaded from the **HAAS** web site at https://www.haascnc.com/it/machines/vertical-mills/vf-series/models/small/vf-1.html and https://www.haascnc.com/it/machines/vertical-mills/vf-series/models/large/vf-11-50.html. The **FreeCAD** software has been needed to translate the step file format into .stl. The latter was reworked in **Blender** software to add colors and to produce precisely  VF-11_50_STEP__7_2022_low_r_03.dae and VF-1_STEP_11_2021_low.dae files, see figures 2.12 and 2.13.

<img src="images/haas_vf-11-50-freecad.png" width="700" />

Figure 2.12: HAAS VF-11-50 CNC machines imported in FreeCAD and exported in stl format file.

<img src="images/haas_vf-11-50-blender.png" width="700" />

Figure 2.13:  HAAS VF-11-50 CNC machines imported into Blender environment to add colors and to export VF-11_50_STEP__7_2022_low_r_03.dae file. 

In [ ]:
<?xml version="1.0" ?>
<model>
    <name>cnc_milling_big</name>
    <version>1.0</version>
    <sdf version="1.6">model.sdf</sdf>
    <author>
        <name>Salvatore Volpe</name>
        <email>salvatore.volpe@mtmt.it</email>
    </author>
    <description></description>
</model>

Code 2.8: model.config file

In [ ]:
<?xml version="1.0" ?>
<sdf version="1.6">
  <model name="cnc_milling_big">
    <link name="link">
        <pose frame=''>0 0 0.0 0 0 0.0</pose>
      <inertial>
        <mass>2</mass>
        <inertia>
          <ixx>0.277</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.452</iyy>
          <iyz>0</iyz>
          <izz>0.268</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <pose frame='link'>0 -1 0 0 0 0</pose>
        <geometry>          
          <box>
            <size>6.5 3 2</size>
          </box>       
         </geometry>
		<surface>
          <friction>
            <ode>
              <mu>0.2</mu>
              <mu2>0.2</mu2>
              <fdir1>0 0 0</fdir1>
              <slip1>0</slip1>
              <slip2>0</slip2>
            </ode>
        </friction>
          </surface>
      </collision>
      <visual name="visual">
	    <geometry>
          <mesh>
            <uri>model://cncmillingbig_model/meshes/VF-11_50_STEP__7_2022_low_r_03.dae</uri>
          </mesh>
        </geometry>
      <meta> <layer> 1 </layer></meta>
</visual>
    </link>
<static>1</static>
  </model>
</sdf>


Code 2.9: model.sdf file

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

#### Chip bin model 

The main characteristics of chip bin model (becket_model) are:
- interact with AMR through reactive surfaces;
- be located through reflective surfaces;
- appearance defined by textures.


It is contained in folders with the following structure:
 
- becket_model
   - materials
       - scripts
        - becket.material
       - textures
        - container_texture.png
        - EuroPallet_Wood.png
   - meshes
     - becket.dae
     - europallet.dae
- model.config
- model.sdf

As regards the first feature, the forks of the AMR hook and lift the europallet. To make this action possible and realistic, collision surfaces are defined using code block 2.10 with which the prismatic shapes are created as shown in figure 2.14

In [ ]:
      <collision name='collision0'>
        <laser_retro>0</laser_retro>
        <max_contacts>10</max_contacts>
        <pose frame=''>0.0 0.0 0.1 0.0 0.0 0.0</pose>
        <geometry>
          <box>
            <size>1.2 0.80 0.04</size>
          </box>
        </geometry>
      </collision>   

Code 2.10: Code block to define a prismatich shape 1.2 x 0.8 x 0.04 [m]

<img src="images/europallet_collisions.png" width="700" />

Figure 2.14: reactive surfaces of the becket_model.  (1) collision 0, (2) collision 3, (3) collision 1, (4) collision 2, (5) collision 4.

<img src="images/europallet_dimensions.png" width="700" />

Figure 2.15: europallet dimensions

About the second feature, four reflective surfaces were placed at the corners of the europallet. The following code creates the geometry and the collision. A new **< laser_retro >** tag reflects the laser rays with an intensity value of 8000. The different value of the reflection intensity allows to locate the position of the corners of the europallet with respect to the laser.

In [ ]:
    <link name='link_reflector_1'>
      <pose frame='link_0'>0.6 -0.35 0.05 0 0 0</pose>
      <gravity>0</gravity>

      <visual name='visual0'>
        <pose frame=''>0 0 0.0 0 0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.1</size></box>
        </geometry>
        <material>
          <lighting>1</lighting>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
      </visual>
      <collision name='collision0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.10</size></box>
        </geometry>
	<laser_retro>8000</laser_retro>
      </collision>
    </link>

Code 2.11: Reflector code

About the appearance, inside the model.sdf file there are two models: the europallet and the bucket (chip bin). Both are represented by means of a .dae CAD file enriched with .png texture files which are respectively contained in the meshes and textures folders. A .material file creates the link between these files.

In [ ]:
      <visual name='europallet'>
        <pose frame=''>0 0.010 0 0 0 1.57</pose>
        <geometry>
          <mesh>
		<uri>model://becket_model/meshes/europallet.dae</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <material>
          <lighting>1</lighting>
          <script>
            <uri>model://becket_model/materials/scripts</uri>
            <uri>model://becket_model/materials/textures</uri>
            <name>becket_model/Diffuse</name>
          </script>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
        <cast_shadows>1</cast_shadows>
        <transparency>0</transparency>
      </visual>

Code 2.12: < scripts > tag to link the .dae file with its texture.

In [ ]:
material becket_model/Diffuse
{
	technique
	{
		pass
		{
			texture_unit
			{
				texture EuroPallet_Wood.jpg
			}
		}
	}
}



Code 2.13: becket.material file

The chip bin and the europallet CADs have been downloaded from https://free3d.com/it/3d-models/pallet in the stl format and, as before, reworked in Blender for the same reasons, see figure 2.16. Next, the .dae file is used in the model.sdf file to represent the appearance.

<img src="images/chip_bins_full_empty_blender.png" width="700" />

Figure 2.16: Chip bins and pallet reworked in Blender.

In [ ]:
<?xml version="1.0" ?>
<model>
    <name>becket_model</name>
    <version>1.0</version>
    <sdf version="1.6">model.sdf</sdf>
    <author>
        <name>Salvatore Volpe</name>
        <email>salvatore.volpe@mtmt.it</email>
    </author>
    <description></description>
</model>

Code 2.14: model.config file

In [ ]:
<?xml version='1.0'?>
<sdf version='1.6'>
  <model name='becket_model'>
    <link name='link_0'>
      <pose frame=''>0 0 0.0 0 -0 0.0</pose>
      <inertial>
         <mass>10</mass>
        <inertia>
          <ixx>0.166667</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.166667</iyy>
          <iyz>0</iyz>
          <izz>0.166667</izz>
        </inertia>
        <pose frame=''>0 0 0 0 -0 0</pose>
      </inertial>
      <gravity>1</gravity>
      <self_collide>0</self_collide>
      <kinematic>0</kinematic>


    <!-- start becket and europallet definitions -->

      <visual name='becket'>
        <pose frame=''>0 0 0.0 0 0 1.57</pose>
        <geometry>
          <mesh>
		        <uri>model://becket_model/meshes/becket.dae</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <!--material>
          <lighting>1</lighting>

          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material-->
        <cast_shadows>1</cast_shadows>
        <transparency>0</transparency>
      </visual>


      <visual name='europallet'>
        <pose frame=''>0 0.010 0 0 0 1.57</pose>
        <geometry>
          <mesh>
		<uri>model://becket_model/meshes/europallet.dae</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <material>
          <lighting>1</lighting>
          <script>
            <uri>model://becket_model/materials/scripts</uri>
            <uri>model://becket_model/materials/textures</uri>
            <name>becket_model/Diffuse</name>
          </script>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
        <cast_shadows>1</cast_shadows>
        <transparency>0</transparency>
      </visual>

  
      <collision name='collision0'>
        <laser_retro>0</laser_retro>
        <max_contacts>10</max_contacts>
        <pose frame=''>0.0 0.0 0.1 0.0 0.0 0.0</pose>
        <geometry>
          <box>
            <size>1.2 0.80 0.04</size>
          </box>
        </geometry>
      </collision>   
    
      <collision name='collision1'>
        <laser_retro>0</laser_retro>
        <max_contacts>5</max_contacts>
        <pose frame='link_0'>0 -0.04 0.03 0 0 0</pose>
        <geometry>
          <box>
             <size>1.0 0.03 0.1</size>
          </box>
        </geometry>
      </collision>

     <collision name='collision2'>
        <laser_retro>0</laser_retro>
        <max_contacts>5</max_contacts>
       <pose frame='link_0'>0 0.04 0.03 0 0 0</pose>
        <geometry>
          <box>
            <size>1.0 0.03 0.1</size>
          </box>
        </geometry>
      </collision>    

      <collision name='collision3'>
        <laser_retro>0</laser_retro>
        <max_contacts>5</max_contacts>
        <pose frame='link_0'>0 0.38 0.03 0 0 0</pose>
        <geometry>
          <box>
            <size>1.0 0.03 0.1</size>
          </box>
        </geometry>
      </collision>

      <collision name='collision4'>
        <laser_retro>0</laser_retro>
        <max_contacts>5</max_contacts>
        <pose frame='link_0'>0 -0.38 0.03 0 -0 0</pose>
        <geometry>
          <box>
            <size>1.0 0.03 0.1</size>
          </box>
        </geometry>
      </collision>
    </link>

  <!-- end becket and europallet definitions -->

    
     <!-- start reflectors definitions -->

    <link name='link_reflector_1'>
      <pose frame='link_0'>0.6 -0.35 0.05 0 0 0</pose>
      <gravity>0</gravity>

      <visual name='visual0'>
        <pose frame=''>0 0 0.0 0 0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.1</size></box>
        </geometry>
	<material>
          <lighting>1</lighting>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
      </visual>
      <collision name='collision0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.10</size></box>
        </geometry>
	<laser_retro>8000</laser_retro>
      </collision>
    </link>
    
    <link name='link_reflector_2'>
      <pose frame='link_0'>0.6 0.35 0.05 0 0 0</pose>
      <gravity>0</gravity>

      <visual name='visual0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
           <box><size>0.005 0.05 0.1</size></box>
        </geometry>
	<material>
          <lighting>1</lighting>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
      </visual>
      <collision name='collision0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.10</size></box>
        </geometry>
	<laser_retro>8000</laser_retro>
      </collision>
    </link>

    <link name='link_reflector_3'>
       <pose frame='link_0'>-0.6 0.35 0.05 0 0 0</pose>
      <gravity>0</gravity>

      <visual name='visual0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
         <box><size>0.005 0.05 0.1</size></box>
        </geometry>
	<material>
          <lighting>1</lighting>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
      </visual>
      <collision name='collision0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.10</size></box>
        </geometry>
	<laser_retro>8000</laser_retro>
      </collision>
    </link>
    
    <link name='link_reflector_4'>
      <pose frame='link_0'>-0.6 -0.35 0.05 0 0 0</pose>
      <gravity>0</gravity>

      <visual name='visual0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.1</size></box>
        </geometry>
	<material>
          <lighting>1</lighting>
          <ambient>0.3 0.3 0.3 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
          <specular>0.01 0.01 0.01 1</specular>
          <emissive>0 0 0 1</emissive>
          <shader type='vertex'>
            <normal_map>__default__</normal_map>
          </shader>
        </material>
      </visual>
      <collision name='collision0'>
        <pose frame=''>0 0 -0.0 0 -0 0</pose>
        <geometry>
          <box><size>0.005 0.05 0.10</size></box>
        </geometry>
	<laser_retro>8000</laser_retro>
      </collision>
    </link>

    

    <joint name='joint_0_to_reflector_1' type='fixed'>
      <parent>link_0</parent>
      <child>link_reflector_1</child>
      <pose frame=''>0 0 0 0 -0 0</pose>
      <physics>
        <ode>
          <limit>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </limit>
          <suspension>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </suspension>
        </ode>
      </physics>
    </joint>
    <joint name='joint_0_to_reflector_2' type='fixed'>
      <parent>link_0</parent>
      <child>link_reflector_2</child>
      <pose frame=''>0 0 0 0 -0 0</pose>
      <physics>
        <ode>
          <limit>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </limit>
          <suspension>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </suspension>
        </ode>
      </physics>
    </joint>
    <joint name='joint_0_to_reflector_3' type='fixed'>
      <parent>link_0</parent>
      <child>link_reflector_3</child>
      <pose frame=''>0 0 0 0 -0 0</pose>
      <physics>
        <ode>
          <limit>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </limit>
          <suspension>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </suspension>
        </ode>
      </physics>
    </joint>
    <joint name='joint_0_to_reflector_4' type='fixed'>
      <parent>link_0</parent>
      <child>link_reflector_4</child>
      <pose frame=''>0 0 0 0 -0 0</pose>
      <physics>
        <ode>
          <limit>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </limit>
          <suspension>
            <cfm>0</cfm>
            <erp>0.2</erp>
          </suspension>
        </ode>
      </physics>
    </joint>

    <static>0</static>
    <allow_auto_disable>1</allow_auto_disable>
  </model>
</sdf>


Code 2.15: model.sdf

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 3. Navigation

In order for the AMR to navigate with ROS it must first create a map of the environment in which the AMR will move. The creation of the map, i.e. the **mapping**, is done using the front laser, set in the urdf file. Next, it needs to find the **location** of the AMR within the map, this process is known as localization. It often talks about **SLAM** (Simultaneous Localization And Mapping), i.e. the two processes together. Once done, it can set up the **navigation stack** which plans a route between a start point and an end point while avoiding both stationary and moving obstacles. 

In short, navigation takes place via the **move_base node** which moves the AMR by imposing velocity commands knowing the position of the target and other information as shown in figure 3.1.

<img src="images/navigation_stack_simple.png" width="700" />

Figure 3.1.: navigation stack simple scheme.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 3.1 amr_mapping

As just mentioned before, to use the navigation stack, it must first create a map. To achieve this, the **gmapping package** is is available. The ROS gmapping package is an implementation of a specific SLAM algorithm called **gmapping**.
In this implementation the gmapping runs on the **amr_mapping package** that has the following structure:

- launch
   - mapping.launch
   - provide_map.launch
- maps
    - mechanical_workshop_map.pgm
    - mechanical_workshop_map.yaml
- params
    - mapping_params.yaml
- src
- CMaleLists.txt
- package.xml

The algorithm runs the **slam_gmapping node** which allows to create a 2-D occupancy grid map (OGM) using laser readings, subscribing to **/front_scan** topic, and odometry pose data, subscribing to **/tf** topic, as input data as the AMR moves through the machine shop environment. To do this, it needs to set up the laser and odometry, from base_footprint and odom frame, respectively in the **mapping.launch** and **mapping_params.yaml** files below.

In [ ]:

     base_frame : "base_footprint"
     odom_frame : "odom"   
     scan_topic : "front_scan"   
     map_update_interval : 5.0
     maxRange : 8.0 
     sigma : 0.05 
     kernelSize : 1 
     lstep : 0.05 
     astep : 0.05 
     iterations : 5 
     lsigma : 0.075 
     ogain : 3.0 
     lskip : 0 
     minimumScore : 200 
     srr : 0.01 
     srt : 0.02 
     str : 0.01 
     stt : 0.02 
     linearUpdate : 0.5 
     angularUpdate : 0.436 
     temporalUpdate : -1.0 
     resampleThreshold : 0.5 
     particles : 80 
     xmin : -10.0
     ymin : -10.0
     xmax : 10.0
     ymax : 10.0
     delta : 0.05 
     llsamplerange : 0.01 
     llsamplestep : 0.01 
     lasamplerange : 0.005 
     lasamplestepvalue : 0.005 
     

Code 3.1: mapping_params.yaml file

In [ ]:
<launch>     
     <node pkg="gmapping" type="slam_gmapping" name="slam_gmapping" output="screen">
     <rosparam file="$(find amr_mapping)/params/mapping_params.yaml" command="load"/> \
     <remap from="/scan" to="/front_scan" />
   
  </node>
</launch>

Code 3.2: mapping.launch file

During the mapping process the slam_mapping node publishes the map in the **/map** topic so that it possible to visualize in RViz as showm in figure 3.2.

<img src="images/mapping.png" width="700" />

Figure 3.2: mapping process of the mechanical workshop.


The /map topic uses a message type of nav_msgs/OccupancyGrid, since it is an OGM. Occupancy is represented as an integer in the range {0, 100}. Cells with a number greater than 0.65 are occupied and those with a number less than 0.196 are free. The Map data is stored in the mechanical_workshop_map.yaml and mechanical_workshop_map.pgm files created inside the map folder as follows:

In [ ]:
cd catkin_ws/src/amr_mapping/maps/
~/catkin_ws/src/amr_mapping/maps$ rosrun map_server map_saver -f mechanical_workshop_map

In [ ]:
image: mechanical_workshop_map.pgm
resolution: 0.050000
origin: [-10.000000, -21.200000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196

# image : Path to the image file containing the occupancy data; can be absolute, or relative to the location of the YAML file
# resolution : Resolution of the map, meters / pixel
# origin : The 2-D pose of the lower-left pixel in the map, as (x, y, yaw), with yaw as counterclockwise rotation (yaw=0 means no rotation). Many parts of the system currently ignore yaw.
# occupied_thresh : Pixels with occupancy probability greater than this threshold are considered completely occupied.
# free_thresh : Pixels with occupancy probability less than this threshold are considered completely free.
# negate : Whether the white/black free/occupied semantics should be reversed (interpretation of thresholds is unaffected) 

Code 3.3: mechanical_workshop_map.yaml

<img src="images/map.png" width="700" />

Figure 3.2: mechanical_workshop_map.pgm file

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 3.2 amr_localization

The **amr_localization package** provides to estimate the pose of the AMR using the **fixed** sensor readings. Setting LaserScan, Map display and PoseArray Display is possible to visualize the localization (pose) of the AMR.

<img src="images/amcl.png" width="700" />

Figure 3.3: amcl algorithm. Arrows inside the purple circle are the output of the amcl.

In this package runs the **amcl node** (Adaptive Monte Carlo Localization) which uses the MCL system in order to track the localization of the AMR moving in a 2D space. This node subscribes to the **/front_laser** topic, the **/map** topic, and the **/tf** topic, and publishes its estimated position (particles i.e. future poses) in to the **/amcl_pose** and the **/particlecloud** topic that is usefull to visualize the arrows in RViz, figure 17. On startup, the amcl node initializes its particle filter according to the parameters provided and with **/initialpose** topic. 

The structure of the amr_localization package is:
- launch
  - amcl.launch
  - initialpose.launch
  - map_warehouse_service_server.launch
  - record_poi_server.launch
- params
  - amr_localization_params.yaml
- rviz_config
  - rviz_config.rviz
- src
  - scripts
     - amr_initialpose.py
     - poi.yaml
     - record_poi_server.py
- srv
  - Poi_message.srv
- CMakeLists.txt
- package.xml  


[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 3.3 amr_navigation

The Navigation Stack is a set of ROS nodes and algorithms which are used to autonomously move a robot from one point to another, avoiding all obstacles. 

<img src="images/navigation_stack.png" width="700" />

Figure 3.4: Navigation stack diagram  (figure from ROS Wiki).

The amr_navigation package contains the **move_base node** that implements an action server that provides to move to the different possible goal positions:
- to a cnc_machines;
- to home;
- to full/empty chip bins;
- to recycling place.

The goal positions are sent through the web panel, see code 3.4 and 3.5, to the GoToPoint server (main_navigation_node) that communicates the relative pose to the move_base node, figure 3.5.

In [ ]:
// calling go_to_point service
callingGotoClients = (request) => {
    var gotoClient = new ROSLIB.Service({
        ros: this.props.ros,
        name : '/go_to_point',
        serviceType : 'amr_localization/Poi_message'
    });

    gotoClient.callService(request, function(result) {

    });
}

Code 3.4: set up the call to /go_to_point with goal position message.

In [ ]:
// call service go_to_point to go to empty_bin
startTask = () => {
    var request = new ROSLIB.ServiceRequest({
        label: 'load_area_bin_2',            
    });
    this.props.callingGotoClients(request)
    console.log("load_area_bin_2")
}

Code 3.5: call the /go_to_point server.

<img src="images/go_to_point.png" width="700" />

Figure 3.5: communication between: move_base, main_navigation node and web page.

The structure of the amr_navigation package is:
- config
  - constmap_common,yaml
  - costmap_exploration.yaml
  - costmap_global_laser.yaml
  - costmap_global_static.yaml
  - costmap_local.yaml
- launch
  - amr_urdf_visualize.launch
  - main.launch
  - navigation.launch
- params
  - amr_move_base_params.yaml
- rviz_config
  - rviz_config.rviz
- src
  - scripts
     - go_to_point_action_clients.py
     - main_navigation.py
- CMakeLists.txt
- package.xml  
  
  
  
  
  
  

  
  


[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

#### Global planner
The move_base node sends the goal pose to the **Navfn global planner** to calculate the shortest path from the current position to the goal position using Dijkstra's algorithm. This path is calculated before the AMR starts moving, so that it will take into account only the **global costmap**. The global cost map is created from the static map and is shown in figure 3.6.

<img src="images/costmap_global.png" width="700" />

Figure 3.6: Global costmap of the mechanical workshop.

Main parameters set in costmap_common.yaml

In [ ]:
footprint: [[0.30, -0.60], [-1.5, -0.60], [-1.5, 0.60], [0.30, 0.60]]
footprint_padding: 0.01 # default value: 0.01

robot_base_frame: "base_link"
update_frequency: 4.0
publish_frequency: 3.0
transform_tolerance: 0.5

resolution: 0.01

#layer definitions
static:
    map_topic: /map
    subscribe_to_updates: true

raytrace_range: 3.0

obstacles_laser:
    observation_sources: front_laser    
    front_laser: {data_type: LaserScan, clearing: true, marking: true, topic: front_scan, inf_is_valid: true, obstacle_range: 5.5}
   
inflation:
    inflation_radius: 2

Code 3.6: parametes of costmap_common.yaml

#### Local planner

The Navfn global planner sent to the DWA (Dynamic Window Approach algorithm) local planner the calculated path to follow.  Then, the local planner executes each segment of the global plan providing velocity commands in order to move the AMR.

The DWA local planner uses the local costmap in order to calculate the local path. The local costmap is  created directly from the AMR's front laser subscribing to **/front_scan**. In this way the AMR can avoid the obstacles.

Main parameters set in costmap_global_static.yaml

In [ ]:
global_frame: map
rolling_window: false
static_map: true

plugins:

  - {name: static,                  type: "costmap_2d::StaticLayer"} # Used to initialize the costmap from a static map.
  - {name: obstacles_laser,         type: "costmap_2d::VoxelLayer"} # Used to add the obstacle sees from laser to the static map
  - {name: inflation,               type: "costmap_2d::InflationLayer"} #  Used to inflate obstacles.

Code 3.7: parametes of costmap_global_static.yaml

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

<img src="images/costmap_local.png" width="700" />

Figure 3.7: Local costmap of the mechanical workshop.

Main parameters set in costmap_local.yaml

In [ ]:
global_frame: odom
rolling_window: true

inflation:
    inflation_radius: 0.85
    cost_scaling_factor: 10
  
plugins:
  - {name: obstacles_laser,           type: "costmap_2d::ObstacleLayer"} 
  - {name: inflation,                 type: "costmap_2d::InflationLayer"}

  

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 4. Chip bin management

The chip bin management regards the cycle in which the AMR provides to substitute the full chip bin with the empty chip bin. To achive this goal the AMR has to perform the following actions:
- detecting the bin
- hooking the bin
- disengaging the bin

#### 4.1 amr_detection

The **amr_detection package** launches the **detect_europallet_node** in which runs the algorithm implemented in the **europallet_detection.py** file, see the structure package below. 


This algorithm, using the rear laser (/rear_scan), provides to stage the detection phase that it is articulated in the following steps:
- finding the reflector points at hight reflection intesities (8000);
- dividing the point in two **Lists**: right reflector and left reflector;
- calculating the average value in both Lists to find the left and the right reflectors;
- calculating the distances between the reflectors and the rear laser;
- calculating the angles of the beams which identifies the reflectors;
- using Carnot theorem to calculate the correction angle;
- calculating the orientation qc and creates a middle frame **ep_frame**, in respect to the rear laser frame, that has the x axis contained in the longitudinal plane of the bin;
- creating a shifted frame **sep_frame** in respect to ep_frame (figure 4.1); 
- transforming the ep_frame and sep_frame from **rear_laserscanner_link** frame to the **odom** frame;


The structure of the amr_detection package is:
- launch
  - europallet_detect_service_client.launch
  - europallet_detect_service_server.launch
- src 
  - amr_detection
    - _ _ init _ _.py
    - europallet_detection.py
    - europallet_detector_client.py
    - europallet_detector_server.py
- srv
   - Pose_message.srv
- CMakeLists.txt
- package.xml
- setup.py

<img src="images/detection_frames.png" width="700" />

Figure 4.1: detection phase: (1) chip bin frame (ep_frame) with x axis on the longitudinal plane of the bin. (2) shifted chip bin frame (sep_frame) with x axis on the longitudinal plane of the bin.

The server **europallet_detector_server.py** publishes through **the Poi_message** the pose of the ep_frame and the sep_frame.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

#### 4.2 amr_rendezvous

The amr_rendezvous package launches the europallet_randezvous_node in which runs the algorithm implemented in the europallet_randezvous.py file, see the structure package below. 

This algorithm provides to stage the approach phase between the AMR and the chip bin in the following steps:
- calling the /europallet_detector_server and writing the result in the goal_delta_pose;
- rotating the AMR until its nose is in line with the y-axis i.e. is orthogonal to the symmetric axis of the europallet footprint;
- moving the AMR along the y-axis until the base_link overlaps the origin of the sep_frame;
- rotating again the AMR until the node is in line with the symmetric axis of the europallet footprint;
- moving forward the AMR in closed loop so that yaw angle remains of 3.14 radians.
- stopping the AMR when it has forked the europallet.


The figure 4.3 shows the algorithm in action.

The structure of the amr_rendezvous package is:
- launch
  - europallet_randezvous_server.launch
- src 
   - europallet_randezvous.py 
   - europallet_randezvous_server.py
- CMakeLists.txt
- package.xml
- setup.py

<img src="images/detection_fork.gif" width="700" />

Figure 4.3: approach europallet algorithm.

Calling the **/randezvous** server, with the **trigger** message, it stages the algorithm.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

#### 4.3 amr_disengagement

The **amr_disengagement package** launches the europallet_disengagement_node in which runs the algorithm implemented in the europallet_disengagement.py file, see the structure package below, that provides to disengages the AMR from the europallet as shown in the figure 4.4.. 

The structure of the amr_disengagement package is:
- launch
  - europallet_disengagement_server.launch
- src 
   - europallet_disengagement.py 
   - europallet_disengagement_server.py
- CMakeLists.txt
- package.xml
- setup.py

<img src="images/disengagement empty bin and go home.gif" width="700" />

Figure 4.4: disengagement phase.

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

#### 4.4 amr_webpage

React components allow you to split your code into reusable and isolated pieces.

The structure of the amr_webpage folder is:
- components
  - actions
   - disengagement_chip_bin.js
   - elevator_down_chip_bin.js
   - elevator_up_chip_bin.js
   - go_home.js
   - goto_cnc_machine.js
   - goto_empty_chip_bin.js
   - goto_full_chip_bin.js
   - manual_control.js
   - move_chip_bin_to_recycling.js
   - randez_vous_chip_bin.js
   - recycling_bin.js
- sensors 
   - laser.js
   - odometry.js
- footer.js
- header.js
- ros_panel.js
- nvm
- app.js
- index.html
- styles.css 


[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 5. Installation

Go to https://github.com/motomechatronics/amr_pallet_trucks

[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)

### 6. Usage 

Open a terminal and type:

In [ ]:
roslaunch amr_description main_project.launch

Open another terminal and type:

In [ ]:
cd ~/catkin_ws/src/amr_pallet_trucks/amr_webpage/

In [ ]:
http-server --port 7000

Open another terminal and type:

In [ ]:
webpage_address

You will get a web link. Copy and paste the link in a browser or clink on the link. After in the same terminal type:

In [ ]:
rosbridge_address

You will get an alphanumeric string, copy and paste the string into the web interface shown in figure 6.1 and press the connect button.

The web interface shown in figure 6.1 allows to command the AMR. In particular, using the interface it is possible to substitute, at the moment, only one chip bin.



<img src="images/webinterface.png" width="700" />

Figure 6.1: Web Interface. Buttons: (1) go to full bin, (2) fork the bin, (3) elevator up, (4) go to recycling, (5) elevator down, (6) disengagement bin, (7) recycling bin, (8) go to empty bin, (9) go to CNC machine, (10) go home, (11) move forward, (12) move backward, (13) move turn CW, (14) move turn CCW, (15) stop, (16) connect/disconnect. Readings: (17) odometry position, (18) obstacle distances.

The procedure to stage a substitution cycle is the following:
- press the **go to full bin** button, the expected result is shown in figure 6.2.

<img src="images/approch_full_bin2.gif" width="700" />

Figure 6.2: result of pressing the go to full bin button.

after that, 
- press the **fork the bin** button and next the **elevator up** button, the expected result is shown in figure 6.3.


<img src="images/fork and lift full bin.gif" width="700" />

Figure 6.3: result of pressing the fork the bin and the elevator up buttons.


- press the **go to recycling** button and next the **elevator down** button, the expected result is shown in figure 6.4.

<img src="images/move to recycling area lift down full bin.gif" width="700" />

Figure 6.4: result of pressing the go to recycling and the elevator down buttons.

- press the **disengagement** button and when the AMR will be disengaged press the **recycling bin** and the **go to empty bin** buttons. The expected result is shown in figure 6.5.

<img src="images/disengagement and go to load empty bin.gif" width="700" />

Figure 6.5: result of pressing  the disengagement bin, the recycling bin and the go to empty bin buttons.

- pressing **fork the bin** button, the expected result is shown in figure 6.6.

<img src="images/fork empty bin and lift up.gif" width="700" />

Figure 6.6: result of pressing the go to empty bin button.

- press the **go to cnc machine** button and next the **elevator down** button, the expected result is shown in figure 6.7.

<img src="images/go to cnc machine and lift down empty bin.gif" width="700" />

Figure 6.7: result of pressing the go to cnc machine and the elevator down buttons.


- by pressing the **disengagement** and the **go home** buttons the cycle ends. The expected result is shown in figure 6.8.

<img src="images/disengagement empty bin and go home.gif" width="700" />

Figure 6.8: result of pressing the disengagement and the go home buttons.


[Go to top](#Autonomous-Mobile-Robot-Pallet-Truck-inside-a-Smart-Factory)